In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

In [2]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [4]:
# add the 'src' directory as one where we can import modules
cwd = os.getcwd()
src_dir = join(cwd, os.pardir, 'src')
sys.path.append(src_dir)

In [26]:
from util.utils import rename_cols

## Load data

In [31]:
#EPA CEMS data
path = join(cwd, '..', 'Data storage', 'Derived data',
            'Monthly EPA emissions 2017-08-31.csv')
epa = pd.read_csv(path)
rename_cols(epa)
epa = epa.groupby(['year', 'month', 'plant id']).sum()

In [6]:
#EIA facility data
path = join(cwd, '..', 'Data storage',
            'Facility gen fuels and CO2 2017-08-31.zip')
eia_fac = pd.read_csv(path)
rename_cols(eia_fac)
eia_fac = eia_fac.groupby(['year', 'month', 'plant id', 'fuel']).sum()

## Find a facility with biomass and CHP

In [10]:
idx = pd.IndexSlice

In [37]:
example_eia = eia_fac.loc[idx[2016, 6, 1897, :], :]
example_eia

total fuel (mmbtu)  generation (mwh)  \
year month plant id fuel                                         
2016 6     1897     NG                 909.0             3.547   
                    SUB              19577.0            76.387   
                    WDS             218373.0           852.066   

                          elec fuel (mmbtu)        lat        lon  quarter  \
year month plant id fuel                                                     
2016 6     1897     NG                 56.0  46.735331 -92.151711        2   
                    SUB              1199.0  46.735331 -92.151711        2   
                    WDS             13372.0  46.735331 -92.151711        2   

                          all fuel fossil co2 (kg)  elec fuel fossil co2 (kg)  \
year month plant id fuel                                                        
2016 6     1897     NG                    48240.63                    2971.92   
                    SUB                 1902884.40                  116542.80   
                    WDS                       0.00                       0.00   

                          all fuel total co2 (kg)  elec fuel total co2 (kg)  
year month plant id fuel                                                     
2016 6     1897     NG                   48240.63                   2971.92  
                    SUB                1902884.40                 116542.80  
                    WDS               20483387.40                1254293.60

In [38]:
example_eia.sum()

total fuel (mmbtu)           2.388590e+05
generation (mwh)             9.320000e+02
elec fuel (mmbtu)            1.462700e+04
lat                          1.402060e+02
lon                         -2.764551e+02
quarter                      6.000000e+00
all fuel fossil co2 (kg)     1.951125e+06
elec fuel fossil co2 (kg)    1.195147e+05
all fuel total co2 (kg)      2.243451e+07
elec fuel total co2 (kg)     1.373808e+06
dtype: float64

In [23]:
example_eia.to_clipboard()

In [35]:
example_epa = epa.loc[idx[2016, 6, 1897], :]
example_epa

co2_mass (kg)         2.311098e+07
gload (mw)                     NaN
heat_input (mmbtu)    2.445270e+05
op_time               7.909800e+02
sload (1000lb/hr)     1.278900e+05
Name: (2016, 6, 1897), dtype: float64

## Adjust CO₂ emissions

In [39]:
co2_factor = (example_eia.sum()['elec fuel fossil co2 (kg)']
              / example_eia.sum()['all fuel total co2 (kg)'])
co2_factor

0.0053272706671432662

In [40]:
co2_factor * example_epa['co2_mass (kg)']

123118.46621921226